In [2]:
import numpy as np
import matplotlib.pyplot as plt
# from IPython.display import Audio
import struct

In [3]:
%%time
from functions import *
import wave
import codecs

CPU times: user 3min 40s, sys: 266 ms, total: 3min 40s
Wall time: 3min 40s


In [4]:
%%time
p = 2 ** 521 - 1
# proof.arithmetic(False)
F = GF(p)
A = p-3
B = 1093849038073734274511112390766805569936207598951683748994586394495953116150735016013708737573759623248592132296706313309438452531591012912142327488478985984
E = EllipticCurve([F(A), F(B)])
q = E.cardinality()

CPU times: user 281 ms, sys: 0 ns, total: 281 ms
Wall time: 287 ms


In [5]:
audio = wave.open(f'audios/original/rick_astley.wav')

In [6]:
data = audio.readframes(audio.getnframes())

In [7]:
data_str = str(data)[2:]

In [8]:
# %%time
P = base_point(E)
print(u'O ponto base é: ')
print(P)

O ponto base é: 
(6803506268652441665320126160432791443663323398140383480423506897862682731335287146943384777282917815923345590490286134867938869348333140091064957929971994539 : 4453799228694375628700582576752017664952796760350127373206270442208169482038182099051399579466443189314701438175562156611484090462679162685766910034348764683 : 1)


In [9]:
# %%time
alice_keys = generate_keys(E, P)

print(u'Chave Privada (Alice): %s \n \n' %(alice_keys[0]))
print(u'Chave Pública (Alice):')
print(alice_keys[1])
print('\n')

Chave Privada (Alice): 2934027355894682609719906250355397518772024858495111899989180015950176712158633740204176132930268433890307560973702243791214838302259403520924586570975594082 
 

Chave Pública (Alice):
(2933191266939757970944617582880093658776558325431749206872221773535175269169790738396625581880909771118433943094678576932114213709887772626057511893129137960 : 92030252321572394067321423942818034695132219268067152036101312713541404714731932510106133283106820521759426978366608598780203812470667686772981160193157221 : 1)




In [10]:
# %%time
bob_keys = generate_keys(E, P)

print(u'Chave Privada (Bob): %s \n \n' %(bob_keys[0]))
print(u'Chave Pública (Bob):')
print(bob_keys[1])
print('\n')

Chave Privada (Bob): 6362658894886954487538962306737022816301184062775702196799413059813207341783996418617553303350822823650001567872983141115514812280696733287012427603058698823 
 

Chave Pública (Bob):
(3306578564397256373982298631542616722724584583493783543800889287198303100953267839485851996525462080965099867865471334572611190001274746485605025080415954520 : 2946094982133288238612062422130922544405488378659931906678467745358072769022740652007311766285916404468122198597778147766044145178541354208087812016748693399 : 1)




In [11]:
%%time
enc_data = encode(E, P, data_str, alice_keys[0], alice_keys[1],bob_keys[1])

CPU times: user 1h 57min 47s, sys: 1min 47s, total: 1h 59min 35s
Wall time: 2h 18min 28s


In [12]:
%%time
dec_data = decode(E, P, enc_data, alice_keys[1], bob_keys[0])

CPU times: user 35min 49s, sys: 25.7 s, total: 36min 15s
Wall time: 36min 15s


In [13]:
%%time
dec_data_wrong1 = decode(E, P, enc_data, alice_keys[1], bob_keys[0] + 1)

CPU times: user 35min 40s, sys: 9.81 s, total: 35min 49s
Wall time: 35min 50s


In [14]:
dec_data_wrong1_b = bytes(dec_data_wrong1, 'utf-8')

In [15]:
# dec_data_wrong1_b = codecs.escape_decode(dec_data_wrong1.encode('utf-8'))[0]

In [16]:
dec_data_b = codecs.escape_decode(dec_data.encode('utf-8'))[0]

In [17]:
dec_data_b = dec_data_b[:-1]

In [18]:
dec_data == data_str

True

In [19]:
dec_data_b == data

True

In [28]:
audio.getframerate()

48000

In [29]:
dec_audio_right = wave.open("audios/decrypted/rick_astley_right.wav", "w")
dec_audio_right.setnchannels(audio.getnchannels())
dec_audio_right.setsampwidth(audio.getsampwidth())
dec_audio_right.setframerate(audio.getframerate())
dec_audio_right.setcomptype('NONE', 'Not Compressed')
dec_audio_right.writeframesraw(dec_data_b)
dec_audio_right.close()

In [30]:
dec_audio_wrong = wave.open("audios/decrypted/rick_astley_wrong.wav", "w")
dec_audio_wrong.setnchannels(audio.getnchannels())
dec_audio_wrong.setsampwidth(audio.getsampwidth())
dec_audio_wrong.setframerate(audio.getframerate())
dec_audio_wrong.setcomptype('NONE', 'Not Compressed')
dec_audio_wrong.writeframesraw(dec_data_wrong1_b)
dec_audio_wrong.close()